In [1]:
# pip install gTTS playsound sounddevice soundfile librosa SpeechRecognition

In [2]:
import os
import random
import datetime
import librosa
import numpy as np
import sounddevice as sd
import soundfile as sf
import speech_recognition as sr
from gtts import gTTS
import playsound

In [3]:
# --- TTS ---
def speak(text):
    print("Ilana:", text)
    tts = gTTS(text=text, lang='en')
    filename = "temp_audio.mp3"
    tts.save(filename)
    playsound.playsound(filename)
    os.remove(filename)

# --- Emotion Classification ---
def classify_tone(y, sr):
    pitch = librosa.yin(y, fmin=50, fmax=300)
    pitch_mean = np.mean(pitch)
    energy = np.mean(librosa.feature.rms(y=y))
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

    if pitch_mean > 180 and energy > 0.02:
        return "Excited / Happy"
    elif pitch_mean < 130 and energy < 0.01:
        return "Calm / Sad"
    elif energy > 0.03:
        return "Angry / Aggressive"
    else:
        return "Neutral / Balanced"

# --- Greeting ---
def greet():
    hour = datetime.datetime.now().hour
    if 5 <= hour < 12:
        speak("Good morning! I'm Ilana.")
    elif 12 <= hour < 17:
        speak("Good afternoon! I'm Ilana.")
    elif 17 <= hour < 21:
        speak("Good evening! I'm Ilana.")
    else:
        speak("Hello! I'm Ilana.")
    speak("I'm here to help you analyze your personality!")

# --- Emotion-based Response ---
def speak_response(emotion):
    responses = {
        "Excited / Happy": [
            "You sound really upbeat! I love that energy!", "You're clearly in a great mood. what's the good news?", "That excitement is contagious!", "Sounds like something amazing just happened!", "I hear your enthusiasm — let's make the most of it!", "Wow, you’re full of energy! Tell me more!", "You're sounding super positive — want to share what’s going on?", "That's a happy tone! I’m all ears!", "With energy like that, you're unstoppable!", "Keep shining — the world needs that enthusiasm!"
        ],

        "Calm / Sad": [
            "You sound peaceful — taking a moment to breathe can be powerful.", "You seem calm right now. I’m here if you need anything.", "It’s okay to slow down. You’re doing just fine.", "You sound a little down. Want to talk about what’s on your mind?", "I hear a bit of sadness. I'm here for you.", "If something's bothering you, I’m listening.", "Tough days happen, but you’re not alone.", "Even quiet moments have meaning — I'm right here with you.", "Whatever you’re feeling is valid. You matter."
        ],

        "Angry / Aggressive": [
            "I can hear the frustration in your voice. Would you like to talk about it?", "It sounds like something upset you. I'm here to listen.", "That energy sounds intense — want to share what’s going on?", "Everyone gets overwhelmed sometimes. You’re not alone.", "I sense some strong emotions. Let’s take a moment together.", "It’s okay to feel angry — let’s figure it out calmly.", "Let’s take a deep breath. I’m here to support you, not to judge.", "Sounds like you’re dealing with a lot right now.", "You seem fired up — maybe we can work through it together?"
        ],

        "Neutral / Balanced": [
            "Everything seems balanced right now, let's keep it going.", "You’re sounding pretty centered. Want to chat or explore something?", "I'm here when you're ready to dive into something.", "You sound clear-headed — how can I support you?", "That was a smooth tone. What would you like to do next?", "All sounds good from my side — what's on your mind?", "You seem relaxed and in control. Need help with anything?"
        ]
    }
    if emotion in responses:
        speak(random.choice(responses[emotion]))
    else:
        speak("Well, that was... something. You good?")

# --- Record User Voice and Process ---
def record_and_classify():
    duration = 8  # seconds
    fs = 23080
    speak("Please speak now. I’ll listen for 8 seconds.")
    print("Recording...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()
    print("Recording complete.")

    y = audio.flatten()
    sf.write("temp.wav", y, fs)

    emotion = classify_tone(y, fs)

    # Use Google SR to recognize text
    r = sr.Recognizer()
    with sr.AudioFile("temp.wav") as source:
        audio_data = r.record(source)

    try:
        text = r.recognize_google(audio_data)
        print("You said:", text)
    except sr.UnknownValueError:
        text = None
        speak("Couldn't understand what you said.")

    os.remove("temp.wav")
    return emotion, text

# --- Main ---
def main():
    greet()
    while True:
        emotion, spoken_text = record_and_classify()
        if emotion:
            speak_response(emotion)
        else:
            speak("I couldn't detect the tone.")
        speak("Do you want to try again? Say yes or no. Or else you want to exit? Say exit")
        r = sr.Recognizer()
        with sr.Microphone() as source:
            audio = r.listen(source)
        try:
            answer = r.recognize_google(audio).lower()
            if 'no' or 'exit' in answer:
                speak("Goodbye! Take care.")
                break
        except:
                speak("I'm Sorry, I didn’t catch that. Let’s continue.")
                continue

if __name__ == "__main__":
    main()

Ilana: Good evening! I'm Ilana.
Ilana: I'm here to help you analyze your personality!
Ilana: Please speak now. I’ll listen for 8 seconds.
Recording...
Recording complete.
You said: ok the good evening very nice to talk to you I am single parent mother
Ilana: It’s okay to feel angry — let’s figure it out calmly.
Ilana: Do you want to try again? Say yes or no. Or else you want to exit? Say exit
Ilana: I'm Sorry, I didn’t catch that. Let’s continue.
Ilana: Please speak now. I’ll listen for 8 seconds.
Recording...
Recording complete.
You said: good evening nice to meet you
Ilana: Everything seems balanced right now, let's keep it going.
Ilana: Do you want to try again? Say yes or no. Or else you want to exit? Say exit
Ilana: I'm Sorry, I didn’t catch that. Let’s continue.
Ilana: Please speak now. I’ll listen for 8 seconds.
Recording...
Recording complete.
You said: hi good evening very glad to meet you
Ilana: I'm here when you're ready to dive into something.
Ilana: Do you want to try agai

KeyboardInterrupt: 

In [ ]:
# r = sr.Recognizer()
# with sr.Microphone() as source:
#     print("Say something...")
#     audio = r.listen(source)
#
# try:
#     print("You said:", r.recognize_google(audio))
# except Exception as e:
#     print("Error:", e)